# Assignment 4

## Main tasks

In [83]:
from utils import fk_symb
import sympy as sp

# (4,4)
Tbase = sp.eye(4)
# (7,4,4)
Ts_q = [Tbase] + fk_symb

In [84]:
from utils import qs_symb
t = sp.symbols('t')

qs = [sp.Function(f'q{i}')(t) for i in range(1,n_link+1)]
Ts = [t.subs(zip(qs_symb,qs)) for t in Ts_q]

In [85]:
# (7,3)
ps = [np.array(t[:3,3].T.tolist())[0] for t in Ts]

In [86]:
from utils import link_lengths
# (6,1)
cs = link_lengths / 2

In [87]:
import numpy as np
# (6,1)
ks = np.array([2,0,0,2,0,2])

In [88]:
# (6,3,3)
Rs = [t[:3,:3] for t in Ts[1:]]
# (6,3)
pcs = [np.array((t[:3,3] + c * t[:3,k]).tolist()) for t,c,k in zip(Ts[1:], cs, ks)]
# (6,4,4)
Tcs = [sp.eye(4) for i in pcs]
for i,r,p in zip(Tcs, Rs, pcs):
    i[:3,:3] = r
    i[:3,3] = p

In [89]:
# (7,1,3)
us = [np.array(r[:3,2].T.tolist())[0] for r in Ts]

In [90]:
r = 1
# (6,)
ms = [1,3,5,2,4,6]
# (6,)
axs = ['x','z','z','x','z','x']

def get_inertia_matrix(idx, axis):
    i1 = 1/12 * ms[idx] * (3 * r**2 + link_lengths[idx]**2)
    i2 = 1/2 * ms[idx] * r**2

    if axis == 'z':
        return np.diag([i1,i1,i2])
    elif axis == 'x':
        return np.diag([i2,i1,i1])
    else:
        return None

# (6,3,3)
Is = [get_inertia_matrix(i,ax) for i,ax in zip(range(6),axs)]

In [91]:
from utils import n_link

# (6,3,6)
Jvs = [None] * n_link
# (6,3,6)
Jws = [None] * n_link
# (6,3)
pcs1 = [pc[0] for pc in pcs]

for i in range(n_link):
    Jv = sp.zeros(3,6)
    Jw = sp.zeros(3,6)
    for k in range(i+1):
        Jv[:3,k] = np.cross(us[k], pcs1[i] - ps[k])
        Jw[:3,k] = us[k]
    
    Jvs[i] = Jv
    Jws[i] = Jw

In [92]:
M = sp.zeros(6,6)
for m,jv,jw,r,i in zip(ms,Jvs,Jws,Rs,Is):
    M += m * jv.T * jv + jw.T * r * i * r.T * jw

In [93]:

Md = sp.diff(M,t)

In [98]:
M_num = np.array(M.subs(zip(qs,[0,1,0,1,0,1])).evalf()).astype(np.float64)

In [99]:

print(M_num)

[[ 4.19552604e+01  2.58358138e+00 -6.93935538e+00 -8.00145477e-01
  -9.49937989e+00  2.02535345e+01]
 [ 2.58358138e+00  3.32065271e+01  2.75389334e+01 -4.25214186e+00
  -1.66640445e+01 -4.04423819e+00]
 [-6.93935538e+00  2.75389334e+01  4.24496279e+01 -4.25214186e+00
  -2.39377210e+01 -1.47717681e+01]
 [-8.00145477e-01 -4.25214186e+00 -4.25214186e+00  7.78189999e+01
   3.40796204e+01  3.83013439e-02]
 [-9.49937989e+00 -1.66640445e+01 -2.39377210e+01  3.40796204e+01
   4.63123297e+01 -3.73773619e-02]
 [ 2.02535345e+01 -4.04423819e+00 -1.47717681e+01  3.83013439e-02
  -3.73773619e-02  4.13199589e+01]]
